## 注意(attention)！开始做前必读项！
所有的代码一定要在这个文件里面编写，不要自己创建一个新的文件
对于提供的数据集，不要改存储地方，也不要修改文件名和内容
不要重新定义函数（如果我们已经定义好的），按照里面的思路来编写。当然，除了我们定义的部分，如有需要可以自行定义函数或者模块
写完之后，重新看一下哪一部分比较慢，然后试图去优化。一个好的习惯是每写一部分就思考这部分代码的时间复杂度和空间复杂度，AI工程是的日常习惯！
这次作业很重要，一定要完成！ 相信会有很多的收获！

# 导入相关的包 


In [2]:
import numpy as np
import pandas as pd
import lightgbm as lgb
from sklearn.model_selection import GridSearchCV
from sklearn import preprocessing
from sklearn.svm import SVC
from sklearn.decomposition import PCA
from sklearn import metrics
import joblib
from bayes_opt import BayesianOptimization
from gensim import models

import jieba
from imblearn.over_sampling import SMOTE


In [3]:
max_length = 500  # 表示样本表示最大的长度,表示降维之后的维度
sentence_max_length = 1500  # 表示句子/样本在降维之前的维度
Train_features3, Test_features3, Train_label3, Test_label3 = [], [], [], []

# 读取训练好的词嵌入向量和文本文件

## KeyedVectors.load_word2vec_format(file path, binary)  
file path: 词向量模型路径  
binary: text format or binary format

In [ ]:
# ToDo
# 通过models.KeyedVectors加载预训练好的embedding
root_path = '/home/user10000440/notespace/src/Embedding/models/'
# fast_embedding = models.KeyedVectors.load(root_path + 'fasttext_model')
fast_embedding = models.KeyedVectors.load_word2vec_format(root_path + 'fasttext_model_wv.bin', binary=True)
w2v_embedding = models.KeyedVectors.load_word2vec_format(root_path + 'w2v_model_wv.bin', binary=True)

print("fast_embedding输出词表的个数{},w2v_embedding输出词表的个数{}".format(
    len(fast_embedding.vocab.keys()), len(w2v_embedding.vocab.keys())))

print("取词向量成功")

In [ ]:
# ToDo
# 读取train_clean.tsv ，test_clean.tsv训练集和测试集文件
# hint: 通过pandas中的read_csv读取数据集
data_path = '~/dataset/图书分类文本数据集/5813f3b1-617d-47ad-87e8-37a773a983af/file/归档/'

train = pd.read_csv(data_path + 'train_clean.csv', sep='\t')
test = pd.read_csv(data_path + 'test_clean.csv', sep='\t')
print("读取数据完成")

In [ ]:
train.head()

In [ ]:
test.head()

将df中的label映射为数字标签并保存到labelIndex列中

In [ ]:
labelName = train.label.unique()  # 全部label列表
labelIndex = list(range(len(labelName)))  # 全部label标签
labelNameToIndex = dict(zip(labelName, labelIndex))  # label的名字对应标签的字典
labelIndexToName = dict(zip(labelIndex, labelName))  # label的标签对应名字的字典
train["labelIndex"] = train.label.map(labelNameToIndex)
test["labelIndex"] = test.label.map(labelNameToIndex)

In [ ]:
labelName

In [ ]:
# 获取 label embedding

# label_cut = [jieba.cut(label.replace('/', '')) for label in labelName]

# global fast_embedding, w2v_embedding

# 每个label对应一个向量，这样做法导致严重过拟合，因为把label信息加入到训练数据里
# label_w2v_embedding_list = []
# for label in label_cut:
#     query = [word for word in label]
#     label_w2v_arr = np.array([w2v_embedding.get_vector(s)
#                          for s in query if s in w2v_embedding.vocab.keys()])
#     label_w2v_embedding_list.append(label_w2v_arr)
    
# def get_label_embedding(label_index):
#     return label_w2v_embedding_list[label_index]

label_cut = [jieba.cut(label.replace('/', '')) for label in labelName]

global fast_embedding, w2v_embedding

label_w2v_embedding_list = []
for label in label_cut:
    query = [word for word in label]
    label_w2v_arr = np.array([w2v_embedding.get_vector(s)
                         for s in query if s in w2v_embedding.vocab.keys()])
    
    result_avg = np.average(label_w2v_arr, axis = 0)
    label_w2v_embedding_list.append([r for r in result_avg])
    
def get_label_embedding():
    return np.array(label_w2v_embedding_list)

# test
# b = get_label_embedding()


In [ ]:
def query_cut(query):
    '''
    函数说明：该函数用于对输入的语句（query）按照空格进行切分
    '''
    # ToDo
    # 第一步：定义一个query_cut函数 将query按空格划分并返回，
    # hint: 通过sqlit对query进行划分
    query_list = query.split(' ')
    return query_list

# 第二步：然后train和test中的每一个样本都调用该函数，
#      并将划分好的样本分别存储到train["queryCut"]和test["queryCut"]中
train["queryCut"] = train.text.apply(query_cut)
test["queryCut"] = test.text.apply(query_cut)
print("切分数据完成")

In [ ]:
train['queryCut'][0:5]

In [ ]:
with open('stopwords.txt', "r") as f:
    #ToDo
    # 第一步：按行读取停用词文件
    stopWords = [line.strip() for line in f.readlines()]


In [ ]:
def rm_stop_word(wordList):
    '''
    函数说明：该函数用于去除输入样本中的存在的停用词
    Return: 返回去除停用词之后的样本
    '''
    # ToDo
    # 第二步：去除每个样本中的停用词并返回新的样本
    new_word_list = []
    for index, word in enumerate(wordList):
        if word not in stopWords:
            new_word_list.append(word)
    return new_word_list
    
train["queryCutRMStopWord"] = train["queryCut"].apply(rm_stop_word)
# dev["queryCutRMStopWord"] = dev["text"].apply(rm_stop_word)
test["queryCutRMStopWord"] = test["queryCut"].apply(rm_stop_word)
print("去除停用词")
print(type(train["queryCutRMStopWord"]))

In [ ]:
train.head()

In [ ]:
# from scipy.signal import sepfir2d

def functor_con2d(embedding_matrix, n, core_num):# 仿2维卷积
    #print("functor_con2d:", embedding_matrix.shape)
    if (embedding_matrix.shape[0] < n):
        print("functor_con2d return None")
        return None
    
    core_size = embedding_matrix.shape[1]
    #core = np.ones(n * core_size) / (n * core_size)
    
    embedding_matrix_one = embedding_matrix.reshape(embedding_matrix.shape[0] * embedding_matrix.shape[1])
    
    result_all = []
    for core_index in range(core_num):         
        core = np.random.rand(core_size * n)        
        result_ = []
        for j in range(embedding_matrix.shape[0] - n + 1):
            start_index = j * embedding_matrix.shape[1]
            end_index = (n * embedding_matrix.shape[1]) + start_index

            embedding_ = (embedding_matrix_one[start_index:end_index])

            #print("embedding:",embedding_)
            #print("core:", core)
            x = np.convolve(embedding_, core, 'valid')
            #print("con:",x)

            result_.append(x[0])
            
        result_all.append(result_)
        
    # print("functor_con2d end")
       
    return result_all

# test
embedding_matrix_test = np.ones(4 * 2).reshape(4, 2)
functor_con2d(embedding_matrix_test, 2, 3)       

In [ ]:
def Find_embedding_with_windows(embedding_matrix):
    '''
    函数说明：`该函数用于获取在大小不同的滑动窗口(k=[2, 3, 4])， 然后进行平均或取最大操作。
    参数说明：
        - embedding_matrix：样本中所有词构成的词向量矩阵
    return: result_list 返回拼接而成的一维词向量
    '''
    
    result_list = []
    # ToDo:
    # 由于之前抽取的特征并没有考虑词与词之间交互对模型的影响，
    # 对于分类模型来说，贡献最大的不一定是整个句子， 可能是句子中的一部分， 如短语、词组等等。 
    # 在此基础上我们使用大小不同的滑动窗口(k=[2, 3, 4])， 然后进行平均或取最大操作。
    k = [2, 3, 4]
    for n in k:
        x = functor_con2d(embedding_matrix, n, 3)
        if x is None:
            result_avg = np.zeros(1)
            result_max = np.zeros(1)
        else:
            result_avg = np.average(x, axis = 1)
            result_max = np.max(x, axis = 1)
        
        result_list = np.concatenate((result_list, result_avg, result_max), axis=0)
        
    return result_list


# test
embedding_matrix_test = np.ones(4 * 2).reshape(4, 2)
result = Find_embedding_with_windows(embedding_matrix_test)
result

In [ ]:
def softmax(x):
    x_exp = np.exp(x)
    #如果是列向量，则axis=0
    x_sum = np.sum(x_exp, axis = 1, keepdims = True)
    s = x_exp / x_sum    
    return s


def Find_Label_embedding(word_matrix, label_embedding):
    '''
    函数说明：获取到所有类别的 label embedding， 与输入的 word embedding 矩阵相乘， 对其结果进行 softmax 运算，
            对 attention score 与输入的 word embedding 相乘的结果求平均或者取最大
            可以参考论文《Joint embedding of words and labels》获取标签空间的词嵌入
    parameters:
    -- example_matrix(np.array 2D): denotes the matrix of words embedding
    -- embedding(np.array 2D): denotes the embedding of all label in data
    return: (np.array 1D) the embedding by join label and word
    '''
    result_embedding=[]
    # To_Do 
    # 第一步：基于consin相似度计算word embedding向量与label embedding之间的相似度
    # 第二步：通过softmax获取注意力分布
    # 第三步：将求得到的注意力分布与输入的word embedding相乘，并对结果进行最大化或求平均
    word_label_matrix = np.dot(word_matrix, label_embedding)
    
    attention_score = softmax(word_label_matrix)
    
    attention_score_average = np.average(attention_score, axis=1)
    result_avg = np.dot(attention_score_average, word_matrix)
     
    attention_score_max = np.max(attention_score, axis=1)
    result_max = np.dot(attention_score_max, word_matrix)
    
   
    result_embedding = np.concatenate((result_embedding, result_avg, result_max), axis=0)
    
    return result_embedding

# test
embedding_matrix_test = np.array([1 for i in range(80)]).reshape([10,8])
label_embedding_test = np.array([1 for i in range(32)]).reshape([8,4])
Find_Label_embedding(embedding_matrix_test, label_embedding_test)


In [ ]:
g_row_index = 0
def sentence2vec(query):
    '''
    函数说明：联合多种特征工程来构造新的样本表示，主要通过以下三种特征工程方法
            第一：利用word-embedding的average pooling和max-pooling
            第二：利用窗口size=2，3，4对word-embedding进行卷积操作，然后再进行max/avg-pooling操作
            第二：利用类别标签的表示，增加了词语和标签之间的语义交互，以此达到对词级别语义信息更深层次的考虑
            另外，对于词向量超过预定义的长度则进行截断，小于则进行填充
    参数说明：
    - query:数据集中的每一个样本
    return: 返回样本经过哦特征工程之后得到的词向量
    '''
    global max_length
    arr = []
    # 加载fast_embedding,w2v_embedding
    global fast_embedding, w2v_embedding
    fast_arr = np.array([fast_embedding.wv.get_vector(s)
                         for s in query if s in fast_embedding.wv.vocab.keys()])
    # 在fast_arr下滑动获取到的词向量
    if len(fast_arr) > 0:
        windows_fastarr = np.array(Find_embedding_with_windows(fast_arr))
        result_attention_embedding = Find_Label_embedding(
            fast_arr, get_label_embedding().T)
    else:# 如果样本中的词都不在字典，则该词向量初始化为0
        # 这里300表示训练词嵌入设置的维度为300
        windows_fastarr = np.zeros(300) 
        result_attention_embedding = np.zeros(300)

    fast_arr_max = np.max(np.array(fast_arr), axis=0) if len(
        fast_arr) > 0 else np.zeros(300)
    fast_arr_avg = np.mean(np.array(fast_arr), axis=0) if len(
        fast_arr) > 0 else np.zeros(300)

    fast_arr = np.hstack((fast_arr_avg, fast_arr_max))
    # 将多个embedding进行横向拼接
    arr = np.hstack((np.hstack((fast_arr, windows_fastarr)),
                     result_attention_embedding))
    global sentence_max_length
    # 如果样本的维度大于指定的长度则需要进行截取或者拼凑,
    result_arr = arr[:sentence_max_length] if len(arr) > sentence_max_length else np.hstack((
        arr, np.zeros(int(sentence_max_length-len(arr)))))
    return result_arr

# def sentence2vec(query_label, query_label):
#     '''
#     函数说明：联合多种特征工程来构造新的样本表示，主要通过以下三种特征工程方法
#             第一：利用word-embedding的average pooling和max-pooling
#             第二：利用窗口size=2，3，4对word-embedding进行卷积操作，然后再进行max/avg-pooling操作
#             第二：利用类别标签的表示，增加了词语和标签之间的语义交互，以此达到对词级别语义信息更深层次的考虑
#             另外，对于词向量超过预定义的长度则进行截断，小于则进行填充
#     参数说明：
#     - query:数据集中的每一个样本
#     return: 返回样本经过哦特征工程之后得到的词向量
#     '''
#     global g_row_index
#     g_row_index += 1
#     if g_row_index % 200 == 0:
#         with open("/home/user10000440/notespace/output/log.txt", 'w') as f:
#             f.write(str(g_row_index))
            
#         print("g_row_index:", g_row_index)
        
#     query = query_label["queryCutRMStopWord"]
#     label_index = query_label["labelIndex"]
    
#     global max_length
#     arr = []
#     # 加载fast_embedding,w2v_embedding
    
#     global fast_embedding, w2v_embedding
    
#     fast_arr = np.array([fast_embedding.get_vector(s)
#                          for s in query if s in fast_embedding.vocab.keys()])
    
#     # 在fast_arr下滑动获取到的词向量
#     if len(fast_arr) > 0:
#         windows_fastarr = np.array(Find_embedding_with_windows(fast_arr))
        
#         # find label embedding
#         label_index_embedding = get_label_embedding()
        
#         result_attention_embedding = Find_Label_embedding(fast_arr, label_index_embedding.T)
        
#     else:# 如果样本中的词都不在字典，则该词向量初始化为0
#         # 这里300表示训练词嵌入设置的维度为300
#         windows_fastarr = np.zeros(300) 
#         result_attention_embedding = np.zeros(300)

#     fast_arr_max = np.max(np.array(fast_arr), axis=0) if len(
#         fast_arr) > 0 else np.zeros(300)
#     fast_arr_avg = np.mean(np.array(fast_arr), axis=0) if len(
#         fast_arr) > 0 else np.zeros(300)

#     fast_arr = np.hstack((fast_arr_avg, fast_arr_max))
#     # 将多个embedding进行横向拼接
#     fast_win_arr = np.hstack((fast_arr, windows_fastarr))
#     arr = np.hstack((fast_win_arr, result_attention_embedding))

#     global sentence_max_length
#     # 如果样本的维度大于指定的长度则需要进行截取或者拼凑,
#     result_arr = arr[:sentence_max_length] if len(arr) > sentence_max_length else np.hstack((
#         arr, np.zeros(int(sentence_max_length-len(arr)))))
#     return result_arr

In [ ]:
# Train_features, Test_features, Train_label, Test_label = Find_Embedding()

In [ ]:
def Dimension_Reduction(Train, Test):
    '''
    函数说明：该函数通过PCA算法对样本进行降维，由于目前维度不是特别搞高 ，可以选择不降维。
    参数说明：
    - Train: 表示训练数据集
    - Test: 表示测试数据集
    Return: 返回降维之后的数据样本
    '''
    global max_length
    #  To_Do 
    # 特征选择，由于经过特征工程得到的样本表示维度很高，因此需要进行降维 max_length表示降维之后的样本最大的维度。
    # 这里通过PCA方法降维
    
    print("Dimension_Reduction Train.shape:", Train.shape)
    pca = PCA(n_components=max_length)
    pca_train = pca.fit_transform(Train) 
    pca_test = pca.fit_transform(Test) 
    print("Dimension_Reduction for PCA Train.shape:", pca_train.shape)
    return pca_train, pca_test

In [ ]:
def Find_Embedding():
    '''
    函数说明：该函数用于获取经过特征工程之后的样本表示
    Return:训练集特征数组(2D)，测试集特征数组(2D)，训练集标签数组（1D）,测试集标签数组（1D）
    '''
    print("获取样本表示中...")
    min_max_scaler = preprocessing.MinMaxScaler()
    Train_features2 = min_max_scaler.fit_transform(
        np.vstack(train["queryCutRMStopWord"].apply(sentence2vec)))
    Test_features2 = min_max_scaler.fit_transform(
        np.vstack(test["queryCutRMStopWord"].apply(sentence2vec)))
    print("获取样本词表示完成")
    # 通过PCA对样本表示进行降维
    Train_features2, Test_features2 = Dimension_Reduction(
        Train=Train_features2, Test=Test_features2)
    Train_label2 = train["labelIndex"]
    Test_label2 = test["labelIndex"]
    
    # 通过上采样的方法从原始数据集中抽样出平衡的数据集
    # X_resampled , y_resampled = SMOTE().fit_sample(Train_features2 , Train_label2)

    print("加载训练好的词向量")
    print("Train_features.shape =", Train_features2.shape)
    print("Test_features.shape =", Test_features2.shape)
    print("Train_label.shape =", Train_label2.shape)
    print("Test_label.shape =", Test_label2.shape)

    return Train_features2, Test_features2, Train_label2, Test_label2

# def Find_Embedding():
#     '''
#     函数说明：该函数用于获取经过特征工程之后的样本表示
#     Return:训练集特征数组(2D)，测试集特征数组(2D)，训练集标签数组（1D）,测试集标签数组（1D）
#     '''
#     print("获取样本表示中...")
#     min_max_scaler = preprocessing.MinMaxScaler()
    
#     print("获取样本表示中 Train_features2...")
#     Train_features2 = min_max_scaler.fit_transform(
#         np.vstack(train[["queryCutRMStopWord", "labelIndex"]].apply(sentence2vec, axis= 1)))
    
#     print("获取样本表示中 Test_features2...")
#     global g_row_index
#     g_row_index = 0
    
#     Test_features2 = min_max_scaler.fit_transform(
#         np.vstack(test[["queryCutRMStopWord", "labelIndex"]].apply(sentence2vec, axis= 1)))
    
#     print("获取样本词表示完成")
#     # 通过PCA对样本表示进行降维
#     Train_features2, Test_features2 = Dimension_Reduction(
#         Train=Train_features2, Test=Test_features2)
#     Train_label2 = train["labelIndex"]
#     Test_label2 = test["labelIndex"]

#     print("加载训练好的词向量")
#     print("Train_features.shape =", Train_features2.shape)
#     print("Test_features.shape =", Test_features2.shape)
#     print("Train_label.shape =", Train_label2.shape)
#     print("Test_label.shape =", Test_label2.shape)

#     return Train_features2, Test_features2, Train_label2, Test_label2


In [ ]:
def Predict(Train_label, Test_label, Train_predict_label, Test_predict_label, model_name):
    '''
    函数说明：直接输出训练集和测试在模型上的准确率
    参数说明：
        - Train_label: 真实的训练集标签（1D）
        - Test_labelb: 真实的测试集标签（1D）
        - Train_predict_label: 模型在训练集上的预测的标签(1D)
        - Test_predict_label: 模型在测试集上的预测标签（1D）
        - model_name: 表示训练好的模型
    Return: None
    '''
    # ToDo
    # 通过调用metrics.accuracy_score计算训练集和测试集上的准确率
    print(model_name+'_'+'Train accuracy %s' % str(np.sum(Train_label == Train_predict_label) / len(Train_label)))
    # 输出测试集的准确率
    print(model_name+'_'+'test accuracy %s' % str(np.sum(Test_label == Test_predict_label) / len(Test_label)))

# ToDo
第一步：完成LGBMClassifier模型中参数  
第二步：根据joblib.dump保存训练好的模型

In [ ]:
def Grid_Train_model(Train_features, Test_features, Train_label, Test_label):
    '''
    函数说明：基于网格搜索优化的方法搜索模型最优参数，最后保存训练好的模型
    参数说明：
        - Train_features: 训练集特征数组（2D）
        - Test_features: 测试集特征数组（2D）
        - Train_label: 真实的训练集标签 (1D)
        - Test_label: 真实的测试集标签（1D）
    Return: None
    '''
    with open("/home/user10000440/notespace/output/log.txt", 'w+') as f:
        f.write("Grid_Train_model begin")
    
    parameters = {
        'max_depth': [5],
        'learning_rate': [0.2],
#         'n_estimators': [100, 500, 1000, 1500, 2000],
#         'min_child_weight': [0, 2, 5, 10, 20],
#         'max_delta_step': [0, 0.2, 0.6, 1, 2],
#         'subsample': [0.6, 0.7, 0.8, 0.85, 0.95],
#         'colsample_bytree': [0.5, 0.6, 0.7, 0.8, 0.9],
#         'reg_alpha': [0, 0.25, 0.5, 0.75, 1],
#         'reg_lambda': [0.2, 0.4, 0.6, 0.8, 1],
#         'scale_pos_weight': [0.2, 0.4, 0.6, 0.8, 1]
    }
    # 定义分类模型列表，这里仅使用LightGBM模型
    models = [
        lgb.LGBMClassifier(),
    ]
    # 遍历模型
    for model in models:
        model_name = model.__class__.  __name__
        global gsearch
        gsearch = GridSearchCV(
            model, param_grid=parameters, scoring='accuracy', cv=3, n_jobs=3)
        gsearch.fit(Train_features, Train_label)
        # 输出最好的参数
        print("Best parameters set found on development set:{}".format(
            gsearch.best_params_))
    
        with open("/home/user10000440/notespace/output/log.txt", 'w+') as f:
            f.write("Best parameters set found on development set:{}".format(gsearch.best_params_))
    
    
        Test_predict_label = gsearch.predict(Test_features)
        Train_predict_label = gsearch.predict(Train_features)
        Predict(Train_label, Test_label,
                Train_predict_label, Test_predict_label, model_name)
    # 保存训练好的模型
    joblib.dump(gsearch, '/home/user10000440/notespace/output/lgb_model.pkl')


主函数,先求训练集和测试集的词向量，然后根据Grid搜索来找到最佳参数的分类模型

In [ ]:
Train_features, Test_features, Train_label, Test_label = Find_Embedding()

In [ ]:
np.savetxt('/home/user10000440/notespace/output/train_features2.txt', Train_features)
np.savetxt('/home/user10000440/notespace/output/Test_features2.txt', Test_features)
np.savetxt('/home/user10000440/notespace/output/Train_label2.txt', Train_label)
np.savetxt('/home/user10000440/notespace/output/Test_label2.txt', Test_label)


In [ ]:
Grid_Train_model(Train_features=Train_features, Test_features=Test_features,Train_label=Train_label, Test_label=Test_label)

In [ ]:
global gsearch

Test_predict_label = gsearch.predict(Test_features)
Train_predict_label = gsearch.predict(Train_features)
Predict(Train_label, Test_label, Train_predict_label, Test_predict_label, "model_name")
# 保存训练好的模型
joblib.dump(gsearch, '/home/user10000440/notespace/output/lgb_model.pkl')

In [ ]:
Test_predict_label = gsearch.predict(Test_features)
Train_predict_label = gsearch.predict(Train_features)
Predict(Train_label, Test_label, Train_predict_label, Test_predict_label, "model_name")